In [8]:
import pandas as pd

# 입력 파일
path = "./cohort/cohort_ver48_remove_trash_col.csv"

# 데이터 로드
df = pd.read_csv(path)

# timestamp로 간주할 키워드
time_keywords = [
    "time", "charttime", "intime", "outtime",
    "starttime", "endtime", "dischtime", "_time"
]

# timestamp 컬럼 자동 탐지
timestamp_cols = [
    col for col in df.columns
    if any(key in col.lower() for key in time_keywords)
]

# subject_id 포함해 최종 남길 컬럼
keep_cols = ["subject_id"] + timestamp_cols

# 중복 제거 (만약 subject_id가 timestamp 탐지에 걸릴 경우 대비)
keep_cols = list(dict.fromkeys(keep_cols))

# 최종 필터링
df_filtered = df[keep_cols]

# 결과 저장 (원하는 경우)
df_filtered.to_csv("./cohort/cohort_ver49_for_petri_net.csv", index=False)

df_filtered.head()


,subject_id,icu_intime,icu_outtime,admittime,first_troponin_charttime,troponin_end_charttime,first_troponin_positive_charttime,first_ecg_charttime,first_stemi_ecg_time,ed_intime,ed_outtime,pci_starttime,pci_endtime,first_antithrombotic_charttime,dischtime
0,10010058,2147-11-18 0:50,2147-11-18 3:19,2147-11-18 2:34,2147-11-18 5:43,2147-11-18 17:42,2147-11-18 5:43,2119-04-03 9:52,2143-10-21 10:40,2147-11-18 0:50,2147-11-18 3:19,2147-11-18 5:34,2147-11-19 8:53,2147-11-18 5:00,2147-11-19 4:00
1,10012438,2178-06-07 19:33,2178-06-07 21:57,2179-07-12 5:56,NaN,NaN,NaN,2167-11-11 13:58,NaN,2178-06-07 19:33,2178-06-07 21:57,2178-06-07 21:45,2178-06-08 15:51,2178-06-07 23:00,2178-06-10 15:30
2,10013310,2153-05-26 8:56,2153-05-26 14:18,2153-06-10 11:55,2153-05-26 14:45,2153-05-26 15:24,2153-05-26 14:45,2118-05-26 8:28,2153-05-06 17:30,2153-05-26 8:56,2153-05-26 14:18,NaN,NaN,2153-05-26 19:00,2153-06-05 19:30
3,10013310,2153-05-06 18:28,2153-05-07 20:47,NaN,2153-05-07 3:13,2153-05-08 10:30,2153-05-07 3:13,2118-05-26 8:28,2153-05-06 17:30,2153-05-06 10:21,2153-05-06 18:28,2153-05-06 18:25,2153-05-06 18:26,2153-05-06 19:00,2153-05-13 13:45
4,10033552,NaN,NaN,NaN,2136-02-29 23:40,2136-03-01 10:31,2136-03-01 7:30,2131-05-04 23:17,NaN,2136-02-29 22:09,2136-03-01 2:26,NaN,NaN,2136-03-01 8:00,2136-03-03 20:30


In [9]:
import pandas as pd

# 1. 파일 로드
path = "./cohort/cohort_ver49_for_petri_net.csv"
df = pd.read_csv(path)

# 2. timestamp/시간 관련 컬럼 자동 탐지
time_keywords = [
    "time", "charttime", "intime", "outtime",
    "starttime", "endtime", "dischtime", "_time"
]

time_cols = [
    col for col in df.columns
    if any(k in col.lower() for k in time_keywords)
]

print("탐지된 시간 컬럼:")
print(time_cols)

# 3. 각 시간 컬럼을 datetime으로 변환 후 포맷 통일
for col in time_cols:
    # 다양한 포맷 허용해서 datetime으로 변환
    df[col] = pd.to_datetime(
        df[col],
        errors="coerce",              # 이상한 값은 NaT로
        infer_datetime_format=True    # 여러 포맷 자동 추론
    )
    # 공통 문자열 포맷으로 통일 (YYYY-MM-DD HH:MM:SS)
    df[col] = df[col].dt.strftime("%Y-%m-%d %H:%M:%S")

# 4. ID 계열도 타입 통일 (있으면)
id_cols = ["subject_id", "hadm_id", "icu_stay_id", "stay_id"]
for col in id_cols:
    if col in df.columns:
        # float로 들어와 있던 거 정수로 변환
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

# 5. 저장
out_path = "./cohort/cohort_ver49_for_petri_net.csv"
df.to_csv(out_path, index=False)

print("저장 완료:", out_path)


탐지된 시간 컬럼:
['icu_intime', 'icu_outtime', 'admittime', 'first_troponin_charttime', 'troponin_end_charttime', 'first_troponin_positive_charttime', 'first_ecg_charttime', 'first_stemi_ecg_time', 'ed_intime', 'ed_outtime', 'pci_starttime', 'pci_endtime', 'first_antithrombotic_charttime', 'dischtime']
저장 완료: ./cohort/cohort_ver49_for_petri_net.csv


C:\Users\User\AppData\Local\Temp\ipykernel_27704\3645472051.py:24: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(
C:\Users\User\AppData\Local\Temp\ipykernel_27704\3645472051.py:24: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(
C:\Users\User\AppData\Local\Temp\ipykernel_27704\3645472051.py:24: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsi

In [10]:
import pandas as pd

# 1. 파일 불러오기
path = "./cohort/cohort_ver49_for_petri_net.csv"
df = pd.read_csv(path)

# 2. timestamp 컬럼 자동 탐지
time_keywords = [
    "time", "charttime", "intime", "outtime",
    "starttime", "endtime", "dischtime", "_time"
]

time_cols = [col for col in df.columns if any(k in col.lower() for k in time_keywords)]

# 3. datetime 변환 (결측 유지)
for col in time_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)

# 4. wide → long
long_df = df.melt(
    id_vars=["subject_id"],
    value_vars=time_cols,
    var_name="activity",
    value_name="timestamp"
)

# 5. timestamp 없는 이벤트 제거
long_df = long_df.dropna(subset=["timestamp"])

# 6. activity 이름 보기 좋게 변경
long_df["activity"] = long_df["activity"].str.upper()
# wide → long
long_df = df.melt(
    id_vars=["subject_id"],
    value_vars=time_cols,
    var_name="activity",
    value_name="timestamp"
)

# timestamp NaN 제거
long_df = long_df.dropna(subset=["timestamp"])

# datetime 변환 통일
long_df["timestamp"] = pd.to_datetime(long_df["timestamp"], errors="coerce")

long_df = long_df.sort_values(["subject_id", "timestamp"])

# 7. 저장
long_df.to_csv("./cohort/cohort_ver49_for_petri_net.csv", index=False)

long_df.head()


C:\Users\User\AppData\Local\Temp\ipykernel_27704\4113045423.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)
C:\Users\User\AppData\Local\Temp\ipykernel_27704\4113045423.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)
C:\Users\User\AppData\Local\Temp\ipykernel_27704\4113045423.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A stri

,subject_id,activity,timestamp
836,10000764,icu_intime,2132-10-14 19:31:00
16276,10000764,ed_intime,2132-10-14 19:31:00
12416,10000764,first_ecg_charttime,2132-10-14 19:40:00
14346,10000764,first_stemi_ecg_time,2132-10-14 19:40:00
2766,10000764,icu_outtime,2132-10-14 23:32:00
